In [115]:
import numpy as np
import pandas as pd
import re
import csv
import json
import requests

# Importing publications

The publications from each facility aare imported.\
The publications were downloaded from the databases of the various facilities.\
For HZB, an API call is used to download the publications\

ESRF/ILL: https://epn-library.esrf.fr/ \
DLS: https://publications.diamond.ac.uk/ \
HZB: https://www.helmholtz-berlin.de/pubbin/publikationen.pl \

Only the ESRF section of the code is complete; the rest are work in progress.

## ESRF

There are four types of ESRF publications. We shall designate them as follows:
<br>
Type 1: Publications with ESRF authors and describing ESRF experiments
<br>
Type 2: Publications without ESRF authors and describing ESRF experiments
<br>
Type 3: Publications with ESRF authors and not describing ESRF experiments
<br>
Type 4: Articles citing the ESRF, no ESRF authors
<br>
This 'Type' information willl be included alongside the other metadata in the dataframes

In [2]:
# Import all ESRF publication files
pub_esrf_type1=pd.read_csv('/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Raw_data/ESRF publications with ESRF authors and describing ESRF experiment (Oct 2024).csv',sep='	', encoding='utf-8',skiprows=1,header=0)    # Import file
pub_esrf_type2=pd.concat([pd.read_csv('/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Raw_data/ESRF Publications without ESRF authors and describing an ESRF experiment (2015 onwards).csv',sep='	', encoding='utf-8',skiprows=1,header=0),pd.read_csv('/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Raw_data/ESRF Publications without ESRF authors and describing an ESRF experiment (bef 2015).csv',sep='	', encoding='utf-8',skiprows=1,header=0)],ignore_index=True)    # Import file
pub_esrf_type3=pd.read_csv('/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Raw_data/ESRF Publications with ESRF authors and not describing an ESRF experiment.csv',sep='	', encoding='utf-8',skiprows=1,header=0)    # Import file
pub_esrf_type4=pd.read_csv('/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Raw_data/ESRF articles citing ESRF, no ESRF author.csv',sep='	', encoding='utf-8',skiprows=1,header=0)    # Import file

# Add 'Type' column
pub_esrf_type1['Type']=1
pub_esrf_type2['Type']=2
pub_esrf_type3['Type']=3
pub_esrf_type4['Type']=4

# Concatenate all four types
pub_esrf=pd.concat([pub_esrf_type1,pub_esrf_type2,pub_esrf_type3,pub_esrf_type4],ignore_index=True)

# Add 'Facility repository' column
pub_esrf['Facility repository']='ESRF'

# Uppercase all the proposal names
pub_esrf['Proposal number'] = pub_esrf['Proposal number'].str.upper()

In [ ]:
# Code to export the processed publications to a CSV file. Uncomment the following lines to run it.

# pub_esrf.to_csv('{insert path here}/Publications_ESRF.csv',index=False)

In [ ]:
# Code to import the processed publications from a CSV file if needed. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Publications_ESRF.csv'
# pub_esrf = pd.read_csv(filepath,encoding='utf-8')

## ILL

There are four types of ILL publications. We shall designate them as follows:
<br>
Type 1: Publications with ILL authors and describing ILL experiments
<br>
Type 2: Publications without ILL authors and describing ILL experiments
<br>
Type 3: Publications with ILL authors and not describing ILL experiments
<br>
Type 4: Articles citing the ILL, no ILL authors
<br>
This 'Type' information willl be included alongside the other metadata in the dataframes

In [ ]:
# Import all ILL publication files
pub_ill_type1=pd.read_csv('{insert path here}/Datasets/ILL/Publications with ILL authors and describing ILL experiment.txt',sep='	', encoding='latin-1',skiprows=1,header=0)    # Import file
pub_ill_type2=pd.read_csv('{insert path here}/Datasets/ILL/Publications without ILL author and describing an ILL experiment.txt',sep='	', encoding='latin-1',skiprows=1,header=0)    # Import file
pub_ill_type3=pd.read_csv('{insert path here}/Datasets/ILL/Publications with ILL authors and not describing ILL experiment.txt',sep='	', encoding='latin-1',skiprows=1,header=0)    # Import file
pub_ill_type4=pd.read_csv('{insert path here}/Datasets/ILL/Articles citing the ILL, no ILL author.txt',sep='	', encoding='latin-1',skiprows=1,header=0)    # Import file

# Add 'Type' column
pub_ill_type1['Type']=1
pub_ill_type2['Type']=2
pub_ill_type3['Type']=3
pub_ill_type4['Type']=4

# Concatenate all four types
pub_ill=pd.concat([pub_ill_type1,pub_ill_type2,pub_ill_type3,pub_ill_type4],ignore_index=True)

# Add 'Facility repository' column
pub_ill['Facility repository']='ILL'

## DLS

In [ ]:
# Import and concat all DLS publications
pub_dls=pd.concat([pd.read_csv('{insert path here}/Datasets/DLS/DLS annual review highlight.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS book chapter.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS conference paper.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS editor note.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS journal paper (2002-2010).csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS journal paper (2011-2020).csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS journal paper (2021-2024).csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS magazine article.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS poster.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS report.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS science highlight.csv'),pd.read_csv('{insert path here}/Datasets/DLS/DLS thesis.csv')],ignore_index=True)

# Add 'Facility repository' column
pub_dls['Facility repository']='DLS'

## HZB

In [487]:
import requests
from bs4 import BeautifulSoup
import os
import time

# Base URL for the form submission
# search_url = "https://www.helmholtz-berlin.de/pubbin/publikationen.pl"
# search_url = "https://www.helmholtz-berlin.de/pubbin/publikationen.pl"

# Years for which to download metadata
years = list(range(1981, 2025))  # Adjust the range of years as needed

# Directory to save metadata
save_directory = 'Datasets/HZB'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Function to extract metadata for a specific year
def extract_metadata_by_year(year):
    print(f"Searching for metadata in year {year}...")
    
    # Form data to submit for the search
    form_data = {
        'jahr': str(year),  # Year
        'JOB': 'start search',  # Start the search
        'sprache':'en',
        'typ_1':'1',
        'typ_2':'1',
        'typ_3':'1',
        'typ_5':'1',
    }

    # Submit the form with a POST request
    response = requests.post(search_url, data=form_data)
    
    if response.status_code != 200:
        print(f"Error: Failed to retrieve results for year {year}")
        return

    # Parse the response HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the metadata section (you'll need to inspect the HTML structure to find the right tag)
    # For example, it might be in a specific <div>, <table>, or other tag
    metadata_text = soup.get_text()  # Get all the text on the page
    # Alternatively, you might want to focus on specific areas using soup.find() or soup.select()

    # Save the metadata to a text file
    metadata_filename = os.path.join(save_directory, f'metadata_{year}.txt')
    with open(metadata_filename, 'w', encoding='utf-8') as f:
        f.write(metadata_text)
    
    print(f"Metadata for year {year} saved to {metadata_filename}")

# Main loop to iterate over years
for year in years:
    extract_metadata_by_year(year)
    time.sleep(5)  # Add a delay between requests


Searching for metadata in year 1981...
Metadata for year 1981 saved to Datasets/HZB/metadata_1981.txt
Searching for metadata in year 1982...
Metadata for year 1982 saved to Datasets/HZB/metadata_1982.txt
Searching for metadata in year 1983...
Metadata for year 1983 saved to Datasets/HZB/metadata_1983.txt
Searching for metadata in year 1984...
Metadata for year 1984 saved to Datasets/HZB/metadata_1984.txt
Searching for metadata in year 1985...
Metadata for year 1985 saved to Datasets/HZB/metadata_1985.txt
Searching for metadata in year 1986...
Metadata for year 1986 saved to Datasets/HZB/metadata_1986.txt
Searching for metadata in year 1987...
Metadata for year 1987 saved to Datasets/HZB/metadata_1987.txt
Searching for metadata in year 1988...
Metadata for year 1988 saved to Datasets/HZB/metadata_1988.txt
Searching for metadata in year 1989...
Metadata for year 1989 saved to Datasets/HZB/metadata_1989.txt
Searching for metadata in year 1990...
Metadata for year 1990 saved to Datasets/HZ

In [652]:
import re

# Function to extract relevant information from a text file
def extract_publications(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Regex pattern to capture text between 'select none' and 'Export as'
    pattern = re.compile(r'select none(.*?)Export as', re.DOTALL)

    # Search for the pattern
    match = pattern.search(content)

    if match:
        # Extract the relevant section
        relevant_section = match.group(1)

        # Clean up the extracted section by stripping unnecessary whitespace
        relevant_section = relevant_section.strip()

        # # Remove multiple blank lines using regex
        # relevant_section = re.sub(r'\n\s*\n+', '\n', relevant_section)

        # Split the cleaned section into lines
        lines = relevant_section.splitlines()

        # publications = []
        # current_entry = []

        # # Process each line to group entries
        # for line in lines:
        #     stripped_line = line.strip()
        #     if stripped_line:  # Only process non-empty lines
        #         current_entry.append(stripped_line)

        #         # Heuristic: Check if the line indicates the end of an entry
        #         # For example, if it contains a DOI or a specific pattern.
        #         if re.search(r'\d{4}', stripped_line) or "doi:" in stripped_line.lower():
        #             # This assumes the entry ends after a line containing a year or DOI
        #             publications.append(" ".join(current_entry))
        #             current_entry = []  # Reset for the next entry

        # # Handle any remaining lines in current_entry
        # if current_entry:
        #     publications.append(" ".join(current_entry))
    else:
        return []
    
    return lines
        

# file_path = 'Datasets/HZB/metadata_2016.txt'  # Replace with your file path
# lines = extract_publications(file_path)




In [653]:
# Loop through all HZB files
publications_hzb=[]
for i in range(1981,2025):
    file_path = 'Datasets/HZB/metadata_'+str(i)+'.txt'  # Replace with your file path
    lines = extract_publications(file_path)
    publications_hzb=publications_hzb+lines

In [656]:
# Extract all DOIs from publications_hzb 
doi_pattern=r"doi:\s(?P<doi>\S+)(?=Open|$)"   # DOI starts with "doi:" followed by a string
hzb_doi=[]
for entry in publications_hzb:
    doi=re.search(doi_pattern,entry)
    if doi:
        hzb_doi.append(doi.group(1))
    else:
        pass

# Get Proposals 
Now, that we have the publications, the next step is to get the proposals.\
For ESRF, this can be done through the API

## ESRF Proposals
The ESRF publications imported above are linked to their corresponding proposals through the proposal number.\
We want to get a complete list of ESRF proposals and map them to the corresponding papers.\
https://icatplus.esrf.fr/api/Documents provides a list of experimental sessions, which can then be mapped to the associated proposals and them to the assosicated publications.\
Note that multiple experimental sessions can be associated with a single proposal.

In [ ]:
# Import ESRF publications

# pub_esrf=pd.read_csv('{insert pathname}/Datasets/ESRF/Publications_ESRF')    # Import file
filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Publications_ESRF.csv'
pub_esrf=pd.read_csv(filepath)    # Import file

# Remove any publications without DOIs
pub_esrf=pub_esrf[pub_esrf['DOI'].notna()]

# Replace any NaN proposal names with empty string
pub_esrf['Proposal number']=pub_esrf['Proposal number'].fillna('')

# Convert all proposal names to uppercase
pub_esrf['Proposal number']=pub_esrf['Proposal number'].str.upper()

In [4]:
# Get list of experimental session documents
api_request="https://icatplus.esrf.fr/api/Documents/"
response=requests.get(api_request).json()
documents_esrf=response

In [5]:
# Extract DOI, Title, and Summary (abstract) information from documents_esrf into session_esrf
# session_esrf will be a list of dicts, with each index in the list representing a proposal; dict keys are: 'doi', 'title', 'summary'

session_esrf=[]        # Initialise empty session_esrf list

for document in documents_esrf:
    session_dict={}        # Initialise empty session_dict dictionary
    
    # Check for DOI and append those with one
    if document['doi']!=None:
        session_dict['doi']=document['doi']
        session_dict['title']=document['title']
        session_dict['summary']=document['summary']
        session_esrf.append(session_dict)
    else:
        pass

In [23]:
# Function to fetch Proposal number information of a single session using API call

def fetch_data(session):
    api_request="https://icatplus.esrf.fr/doi/"
    response = requests.get(api_request+session['doi']+'/reports')

    # Check for valid API call status code
    if response.status_code==200:
        reports_esrf=response.json()
        proposals=[]
        for report in reports_esrf:
            proposals.append({report['proposal']:report['reports']})
        session['proposal_dict']=proposals
        return session     # prop with proposal number added 
    else:
        return 0        # return 0 is API call result is invalid

def fetch_subject(session):
    api_request="https://icatplus.esrf.fr/doi/"
    response2 = requests.get(api_request+session['doi']+'/json-datacite')

    # Check for valid API call status code
    if response2.status_code==200:
        reports2_esrf=response2.json()
        for subject in reports2_esrf['subjects']:
            if (subject['subjectScheme'] == 'Proposal Type Description') & ('subject' in subject):     # Check if the proposal type description exists, AKA the scientific disciplines such as 'Life Sciences' etc.
                session['subject']=subject['subject']
            elif (subject['subjectScheme'] == 'Instrument') & ('subject' in subject):     # Check if the instrument exists, AKA the beamline name
                session['instrument']=subject['subject']
            else:
                pass
        return session
    else:
        return 0

In [7]:
# Loop through each session in session_esrf and apply the fetch_data function. Only append session that has valid proposal number to session_esrf_valid

session_esrf_valid=[]
for session in session_esrf:
    fetched_data=fetch_data(session)
    if fetched_data!=0:
        session_esrf_valid.append(session)
    else:
        pass

In [24]:
# Loop through each session in session_esrf_valid and apply the fetch_subject function.

session_esrf_valid_with_subj=[]
for session in session_esrf_valid:
    fetched_data=fetch_subject(session)
    if fetched_data!=0:
        session_esrf_valid_with_subj.append(session)
    else:
        pass

In [ ]:
# Code to export the processed sessions with metadata to a JSON file. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/session_esrf_valid_with_subj.json'
# with open(filepath, 'w') as f:
#     json.dump(session_esrf_valid_with_subj, f, indent=2)

In [ ]:
# Load JSON file containing processed sessions as a dictionary if needed

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/session_esrf_valid_with_subj.json'    # <--- INSERT YOUR FILEPATH HERE
# with open(filepath, 'r') as f:
#         session_esrf_valid_with_subj = json.load(f)

In [141]:
# Check how many proposals numbers each session is linked to
sum_of_none = 0
sum_of_one = 0
sum_of_more = 0
for session in session_esrf_valid_with_subj:
    if len(session['proposal_dict']) == 1:
        sum_of_one += 1
    elif len(session['proposal_dict']) == 0:
        sum_of_none += 1
    else:
        sum_of_more += 1

print('Number of session with zero proposals',sum_of_none)
print('Number of session with exactly one proposal',sum_of_one)
print('Number of session with more than one proposals',sum_of_more)

Number of session with zero proposals 0
Number of session with exactly one proposal 8348
Number of session with more than one proposals 0


Seems like all sessions are linked to only one proposal. In that case, we can separate the proposal number from the pdf document name for each session.\
For example, 'proposal_dict': [{'MA-6393': ['109865_0001.pdf']}] can be split into 'proposal': 'MA-6393' and 'pdf document name': '109865_0001.pdf'


In [142]:
for session in session_esrf_valid_with_subj:
    session['proposal'] = list(session['proposal_dict'][0].keys())[0]
    
for session in session_esrf_valid_with_subj:
    session['pdf_document_name'] = list(session['proposal_dict'][0].values())[0]

In [143]:
# Convert session_esrf_valid to DataFrame, apply upper case to all proposal names for standardisation, replace all NaN values with blank strings in the 'subject' column.

df=pd.DataFrame(session_esrf_valid_with_subj)
df['proposal']=df['proposal'].str.upper()
df['subject']=df['subject'].fillna('')


# Group by proposal, and for each of the 'summary', 'title', 'subject' columns, concatenate unique strings. For 'doi' column, gather all the experiment session DOIs into a list for each proposal.
df = df.groupby('proposal').agg({
    'summary': lambda x: '. '.join(sorted(set(v for v in x if v))),
    'title': lambda x: '. '.join(set(x)),
    'subject': lambda x: ', '.join(sorted(set(v for v in x if v))),
    'instrument': lambda x: list(sorted(set(v for v in x if v))),
    'doi': lambda x: list(x.dropna()),
    'pdf_document_name': lambda x: list(set(i for sublist in x for i in sublist if i))
}).reset_index()

df.rename(columns={'doi':'experiment_session_doi'},inplace=True)

In [144]:
df

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name
0,A01-2-1247,The proposal falls within the general research...,Crystal structure of multiferroic KNi_1-xCo_xP...,,[BM01],[10.15151/ESRF-ES-670011307],[]
1,A01-2-1248,Having achieved successful results with metal-...,Metal-organic and covalent organic polyhedra f...,,[BM01],[10.15151/ESRF-ES-670011305],[]
2,A01-2-1249,The overall aim of the project is deciphering ...,The redox structure of haem- and flavoproteins...,,[BM01],[10.15151/ESRF-ES-670011413],[98064_A.pdf]
3,A01-2-1254,"In this study, we will investigate structural ...","Nickelates – phase transitions, distortions an...",,[BM01],[10.15151/ESRF-ES-748027553],[]
4,A01-2-1255,We developed a crystallization strategy that p...,Understanding the structure of two-dimensional...,,[BM01],[10.15151/ESRF-ES-670011338],[]
...,...,...,...,...,...,...,...
5379,XA-11,Steel slag is one of the most common wastes pr...,Time-resolved X-ray Tomography imaging and Ram...,,[BM05],[10.15151/ESRF-ES-1647778275],[]
5380,XA-5,The aim of this proposal is to elucidate the i...,ReMade Proposal\r\nImpact of metals blend and ...,,[ID22],[10.15151/ESRF-ES-1424924468],[]
5381,XA-6,Recycling spent Li-ion batteries has attracted...,ReMade Proposal\r\nOperando investigation stru...,,[ID31],[10.15151/ESRF-ES-1436201044],[]
5382,XA-7,This proposals combines our expertise in metal...,ReMade Proposal\r\nTuning the sorption propert...,,[ID31],[10.15151/ESRF-ES-1352264747],[]


In [ ]:
# Code to export the DataFrame with no references/publications metadata to a JSON file. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_base.json'
# df.to_json(filepath, indent=2, orient='records')


### Proposals with PDF metadata only
Map PDF metadata obtained from GROBID to proposals. \
The PDF metadata we are interested in are the main body of text describing the proposal in greater detail, and the referenced works. \
Refer to Grobid.ipynb to see how PDF metadata was obtained.

In [151]:
# Load json file containing proposals in base format
filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_base.json'    # <--- INSERT YOUR FILEPATH HERE
df = pd.read_json(filepath)

In [152]:
# Initlialise empty lists for 'referenced_works DOI' and 'openalex_ids' columns
df['referenced_works_doi'] = np.empty((len(df), 0)).tolist()
df['openalex_ids'] = np.empty((len(df), 0)).tolist()

In [153]:
df

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name,referenced_works_doi,openalex_ids
0,A01-2-1247,The proposal falls within the general research...,Crystal structure of multiferroic KNi_1-xCo_xP...,,[BM01],[10.15151/ESRF-ES-670011307],[],[],[]
1,A01-2-1248,Having achieved successful results with metal-...,Metal-organic and covalent organic polyhedra f...,,[BM01],[10.15151/ESRF-ES-670011305],[],[],[]
2,A01-2-1249,The overall aim of the project is deciphering ...,The redox structure of haem- and flavoproteins...,,[BM01],[10.15151/ESRF-ES-670011413],[98064_A.pdf],[],[]
3,A01-2-1254,"In this study, we will investigate structural ...","Nickelates – phase transitions, distortions an...",,[BM01],[10.15151/ESRF-ES-748027553],[],[],[]
4,A01-2-1255,We developed a crystallization strategy that p...,Understanding the structure of two-dimensional...,,[BM01],[10.15151/ESRF-ES-670011338],[],[],[]
...,...,...,...,...,...,...,...,...,...
5379,XA-11,Steel slag is one of the most common wastes pr...,Time-resolved X-ray Tomography imaging and Ram...,,[BM05],[10.15151/ESRF-ES-1647778275],[],[],[]
5380,XA-5,The aim of this proposal is to elucidate the i...,ReMade Proposal\r\nImpact of metals blend and ...,,[ID22],[10.15151/ESRF-ES-1424924468],[],[],[]
5381,XA-6,Recycling spent Li-ion batteries has attracted...,ReMade Proposal\r\nOperando investigation stru...,,[ID31],[10.15151/ESRF-ES-1436201044],[],[],[]
5382,XA-7,This proposals combines our expertise in metal...,ReMade Proposal\r\nTuning the sorption propert...,,[ID31],[10.15151/ESRF-ES-1352264747],[],[],[]


In [154]:
# Load json file containing PDF metadata as a dictionary
filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/PDF_metadata.json'    # <--- INSERT YOUR FILEPATH HERE
with open(filepath, 'r') as f:
        pdf_dict_processed = json.load(f)

In [155]:
# To concatenate the list of strings in the 'text' field for each entry in `pdf_dict_processed` and replace the list with the concatenated string
for pdf in pdf_dict_processed:
    text_list = pdf_dict_processed[pdf].get('text', [])
    pdf_dict_processed[pdf]['text'] = ' '.join(filter(None, text_list))
    ls=[]
    for open_alex_id in pdf_dict_processed[pdf]['OpenAlex']:
        open_alex_id = 'https://openalex.org/' + open_alex_id
        ls.append(open_alex_id)
    pdf_dict_processed[pdf]['OpenAlex'] = ls

In [156]:
# Map each proposal to the corresponding PDF document names, concactenate all the referenced work DOIs, corresponsding OpenAlex IDs, and text from 
# the PDF metadata into the respective columns of the proposal DataFrame. Make sure the lists of DOIs and OpenAlex IDs do not have repeating elements.
prop_esrf_dict = df.to_dict(orient='records')
prop_esrf_pdf_only = []
for prop in prop_esrf_dict:
    for pdf in prop['pdf_document_name']:
        pdf = pdf.replace('.pdf', '')
        pdf_metadata = pdf_dict_processed.get(pdf)
        if pdf_metadata:
            prop['referenced_works_doi'] = list(set(prop['referenced_works_doi'] + pdf_metadata['DOI']))
            prop['openalex_ids'] = list(set(prop['openalex_ids'] + pdf_metadata['OpenAlex']))
            # prop['referenced_works DOI'] = pdf_metadata['DOI']
            # prop['openalex_ids'] = pdf_metadata['OpenAlex']
            prop['summary'] = prop['summary'] + ' ' + pdf_metadata['text']
    prop_esrf_pdf_only.append(prop)

In [ ]:
# Code to export the processed proposals with PDF metadata to a JSON file. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_PDF_metadata_only.json'
# with open(filepath, 'w') as f:
#     json.dump(prop_esrf_pdf_only, f, indent=2)

### Proposals with publications only
Map proposals to publications.

In [158]:
# Load json file containing proposals in base format
filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_base.json'    # <--- INSERT YOUR FILEPATH HERE
df = pd.read_json(filepath)

In [159]:
# Use Regex to map Proposal names in df to Proposal names in pub_esrf, and get a list of publication DOIs for each proposal
doi_list=[]
for proposal in df['proposal']:
    matching_dois=pub_esrf[pub_esrf['Proposal number'].str.contains(r'\b' + proposal + r'\b', regex=True,na=False)]['DOI'].dropna().to_list()
    doi_list.append(matching_dois)

# Append the list of publication DOIs to df under the new column 'publications DOI'
df['publications_doi']=doi_list

In [160]:
df

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name,publications_doi
0,A01-2-1247,The proposal falls within the general research...,Crystal structure of multiferroic KNi_1-xCo_xP...,,[BM01],[10.15151/ESRF-ES-670011307],[],[]
1,A01-2-1248,Having achieved successful results with metal-...,Metal-organic and covalent organic polyhedra f...,,[BM01],[10.15151/ESRF-ES-670011305],[],[]
2,A01-2-1249,The overall aim of the project is deciphering ...,The redox structure of haem- and flavoproteins...,,[BM01],[10.15151/ESRF-ES-670011413],[98064_A.pdf],[]
3,A01-2-1254,"In this study, we will investigate structural ...","Nickelates – phase transitions, distortions an...",,[BM01],[10.15151/ESRF-ES-748027553],[],[]
4,A01-2-1255,We developed a crystallization strategy that p...,Understanding the structure of two-dimensional...,,[BM01],[10.15151/ESRF-ES-670011338],[],[10.1038/s41563-023-01669-z]
...,...,...,...,...,...,...,...,...
5379,XA-11,Steel slag is one of the most common wastes pr...,Time-resolved X-ray Tomography imaging and Ram...,,[BM05],[10.15151/ESRF-ES-1647778275],[],[]
5380,XA-5,The aim of this proposal is to elucidate the i...,ReMade Proposal\r\nImpact of metals blend and ...,,[ID22],[10.15151/ESRF-ES-1424924468],[],[]
5381,XA-6,Recycling spent Li-ion batteries has attracted...,ReMade Proposal\r\nOperando investigation stru...,,[ID31],[10.15151/ESRF-ES-1436201044],[],[]
5382,XA-7,This proposals combines our expertise in metal...,ReMade Proposal\r\nTuning the sorption propert...,,[ID31],[10.15151/ESRF-ES-1352264747],[],[]


In [ ]:
# Code to export DataFrame to a JSON file. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_publications_only_doi.json'    # <--- INSERT YOUR FILEPATH HERE
# df.to_json(filepath, orient='records', indent=2)

#### Get OpenAlex citation metadata for proposals
Now that we have the proposals with the DOIs of the publications linked to the proposals, we want to use the OpenAlex API call on the DOIs to get the corresponding OpenAlex IDs. This is because the topic classification model takes in OpenAlex IDs and not DOIs as input features.

In [162]:
# Import ESRF proposals with publication DOIs only
filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_publications_only_doi.json'   # <--- INSERT YOUR FILEPATH HERE
prop_esrf=pd.read_json(filepath)

In [163]:
prop_esrf

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name,publications_doi
0,A01-2-1247,The proposal falls within the general research...,Crystal structure of multiferroic KNi_1-xCo_xP...,,[BM01],[10.15151/ESRF-ES-670011307],[],[]
1,A01-2-1248,Having achieved successful results with metal-...,Metal-organic and covalent organic polyhedra f...,,[BM01],[10.15151/ESRF-ES-670011305],[],[]
2,A01-2-1249,The overall aim of the project is deciphering ...,The redox structure of haem- and flavoproteins...,,[BM01],[10.15151/ESRF-ES-670011413],[98064_A.pdf],[]
3,A01-2-1254,"In this study, we will investigate structural ...","Nickelates – phase transitions, distortions an...",,[BM01],[10.15151/ESRF-ES-748027553],[],[]
4,A01-2-1255,We developed a crystallization strategy that p...,Understanding the structure of two-dimensional...,,[BM01],[10.15151/ESRF-ES-670011338],[],[10.1038/s41563-023-01669-z]
...,...,...,...,...,...,...,...,...
5379,XA-11,Steel slag is one of the most common wastes pr...,Time-resolved X-ray Tomography imaging and Ram...,,[BM05],[10.15151/ESRF-ES-1647778275],[],[]
5380,XA-5,The aim of this proposal is to elucidate the i...,ReMade Proposal\r\nImpact of metals blend and ...,,[ID22],[10.15151/ESRF-ES-1424924468],[],[]
5381,XA-6,Recycling spent Li-ion batteries has attracted...,ReMade Proposal\r\nOperando investigation stru...,,[ID31],[10.15151/ESRF-ES-1436201044],[],[]
5382,XA-7,This proposals combines our expertise in metal...,ReMade Proposal\r\nTuning the sorption propert...,,[ID31],[10.15151/ESRF-ES-1352264747],[],[]


In [164]:
from pyalex import Works, config
config.email = 'terence.tan@wadham.ox.ac.uk'

In [165]:
# Function for applying the API call to each row of the proposal DataFrame prop_esrf

def get_openalex_id(row):
    # Get the 'publications DOI' column from the row
    publications_doi = row['publications_doi']
    openalex_id = []
    
    if len(publications_doi) != 0:
        for doi in publications_doi:
            result = Works().filter(doi=doi).select('id').get()
            # If the result is not empty, append the 'id', otherwise do nothing
            if result:
                openalex_id.append(result[0]['id'])
            else:
                pass
    else:
        pass
    
    row['openalex_ids']=openalex_id
   
    return row

In [166]:
# Apply the get_openalex_id to each row of prop_esrf
prop_esrf = prop_esrf.apply(get_openalex_id, axis=1)

In [167]:
# Sanity check
# Check if the number of publications DOI is equal to the number of OpenAlex IDs for each proposal
prop_esrf[prop_esrf['publications_doi'].apply(len)!=prop_esrf['openalex_ids'].apply(len)]

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name,publications_doi,openalex_ids
3512,LS-3076,Pathogenic mechanisms of absestos-related dise...,Release of metals and dissolution of mineral f...,Life Sciences,[ID21],[10.15151/ESRF-ES-744175308],[],[10.13133/2239-1002/18090],[]


There is a mismatch for one of the proposals. This particular publication DOI (10.13133/2239-1002/18090) does not exist in the OpenAlex data repository.

In [ ]:
# Export the proposals with publications only (now with OpenAlex IDs) to a JSON file. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_publications_only.json'    # <--- INSERT YOUR FILEPATH HERE
# prop_esrf.to_json(filepath, orient='records', indent=2)

### Proposals with both PDF metadata and publications
Combine the two proposals

In [169]:
# Import both files and concatenate them
filepath1 = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_PDF_metadata_only.json'   # <--- INSERT YOUR FILEPATH HERE
filepath2 = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_publications_only.json'   # <--- INSERT YOUR FILEPATH HERE
df1=pd.read_json(filepath1)
df2=pd.read_json(filepath2)

In [170]:
df1

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name,referenced_works_doi,openalex_ids
0,A01-2-1247,The proposal falls within the general research...,Crystal structure of multiferroic KNi_1-xCo_xP...,,[BM01],[10.15151/ESRF-ES-670011307],[],[],[]
1,A01-2-1248,Having achieved successful results with metal-...,Metal-organic and covalent organic polyhedra f...,,[BM01],[10.15151/ESRF-ES-670011305],[],[],[]
2,A01-2-1249,The overall aim of the project is deciphering ...,The redox structure of haem- and flavoproteins...,,[BM01],[10.15151/ESRF-ES-670011413],[98064_A.pdf],[],[]
3,A01-2-1254,"In this study, we will investigate structural ...","Nickelates – phase transitions, distortions an...",,[BM01],[10.15151/ESRF-ES-748027553],[],[],[]
4,A01-2-1255,We developed a crystallization strategy that p...,Understanding the structure of two-dimensional...,,[BM01],[10.15151/ESRF-ES-670011338],[],[],[]
...,...,...,...,...,...,...,...,...,...
5379,XA-11,Steel slag is one of the most common wastes pr...,Time-resolved X-ray Tomography imaging and Ram...,,[BM05],[10.15151/ESRF-ES-1647778275],[],[],[]
5380,XA-5,The aim of this proposal is to elucidate the i...,ReMade Proposal\r\nImpact of metals blend and ...,,[ID22],[10.15151/ESRF-ES-1424924468],[],[],[]
5381,XA-6,Recycling spent Li-ion batteries has attracted...,ReMade Proposal\r\nOperando investigation stru...,,[ID31],[10.15151/ESRF-ES-1436201044],[],[],[]
5382,XA-7,This proposals combines our expertise in metal...,ReMade Proposal\r\nTuning the sorption propert...,,[ID31],[10.15151/ESRF-ES-1352264747],[],[],[]


In [171]:
df2

,proposal,summary,title,subject,instrument,experiment_session_doi,pdf_document_name,publications_doi,openalex_ids
0,A01-2-1247,The proposal falls within the general research...,Crystal structure of multiferroic KNi_1-xCo_xP...,,[BM01],[10.15151/ESRF-ES-670011307],[],[],[]
1,A01-2-1248,Having achieved successful results with metal-...,Metal-organic and covalent organic polyhedra f...,,[BM01],[10.15151/ESRF-ES-670011305],[],[],[]
2,A01-2-1249,The overall aim of the project is deciphering ...,The redox structure of haem- and flavoproteins...,,[BM01],[10.15151/ESRF-ES-670011413],[98064_A.pdf],[],[]
3,A01-2-1254,"In this study, we will investigate structural ...","Nickelates – phase transitions, distortions an...",,[BM01],[10.15151/ESRF-ES-748027553],[],[],[]
4,A01-2-1255,We developed a crystallization strategy that p...,Understanding the structure of two-dimensional...,,[BM01],[10.15151/ESRF-ES-670011338],[],[10.1038/s41563-023-01669-z],[https://openalex.org/W4386923995]
...,...,...,...,...,...,...,...,...,...
5379,XA-11,Steel slag is one of the most common wastes pr...,Time-resolved X-ray Tomography imaging and Ram...,,[BM05],[10.15151/ESRF-ES-1647778275],[],[],[]
5380,XA-5,The aim of this proposal is to elucidate the i...,ReMade Proposal\r\nImpact of metals blend and ...,,[ID22],[10.15151/ESRF-ES-1424924468],[],[],[]
5381,XA-6,Recycling spent Li-ion batteries has attracted...,ReMade Proposal\r\nOperando investigation stru...,,[ID31],[10.15151/ESRF-ES-1436201044],[],[],[]
5382,XA-7,This proposals combines our expertise in metal...,ReMade Proposal\r\nTuning the sorption propert...,,[ID31],[10.15151/ESRF-ES-1352264747],[],[],[]


In [190]:
# Rename the 'openalex_ids' columns in both DataFrames to avoid confusion, and then concatenate them
df1.rename(columns={'openalex_ids':'referenced_works_openalex_ids'},inplace=True)
df2.rename(columns={'openalex_ids':'publications_openalex_ids'},inplace=True)

df_combined = df1.merge(df2[['proposal','publications_doi','publications_openalex_ids']], on='proposal', how='inner')

In [195]:
df_combined['combined_openalex_ids'] = df_combined.apply(lambda row: set(row['referenced_works_openalex_ids'] + row['publications_openalex_ids']), axis=1)

In [ ]:
# Export the proposals with combined publications and referenced work Openalex IDs to a JSON file. Uncomment the following lines to run it.

# filepath = '/Users/fdp54928/Library/CloudStorage/OneDrive-Nexus365/GitHub Repositories/synchrotron-proposals-topic-classification/Datasets/ESRF/Checkpoint_data/Proposals_ESRF_combined.json'    # <--- INSERT YOUR FILEPATH HERE
# df_combined.to_json(filepath, orient='records', indent=2)